In [5]:
def u3D(r,z,E=100e6,nu=0.3,F=100e3):
    Rad = np.sqrt(r**2 + z**2)
    G = E/(2*(1+nu))
    fak = F/(4*np.pi*G*Rad)
    return fak * (2*(1-nu) + (z/Rad)**2)

In [6]:
def u1D(r,z,E=100e6,nu=0.3,F=100e3):
    Rad = np.sqrt(r**2 + z**2)
    Es = E*(1-nu)/((1+nu)*(1-2*nu))
    fak = F/(2*np.pi*Es*Rad)
    return fak * (2 + (z/Rad)**2)

In [ ]:
from ipywidgets import widgets
from ipywidgets import interact
def interactive_point_settlement():
    @interact(#c=widgets.BoundedFloatText(value=0.3,min=0, description='$c$ / MPa'),
               E=widgets.BoundedFloatText(value=100, min=1e-3, max=1e3, description='$E$ / MPa'),
               F=widgets.BoundedFloatText(value=100, min=1, max=1e4, description='$F_z$ / kN'),
               nu=widgets.BoundedFloatText(value=0.3, min=0, max=0.49, description='$\\nu$')
    )
    def plot_MC_states(E,F,nu):
        E *= 1e6
        F *= 1e3
        z = np.array([0.1,0.5,1,2,3,5]) # m
        r = np.linspace(0,5,100)
        sns.set_palette("Paired",n_colors=12)
        fig, ax = plt.subplots(figsize=(18,8),ncols=2)
        for i in z:
            ax[0].plot(r,-u3D(r,i,E,nu,F)*1e3-i,label="$z = %.2f$ m" %i)
            ax[0].plot(r,-u1D(r,i,E,nu,F)*1e3-i,label="$z = %.2f$ m" %i,ls='--')
            ax[0].axhline(-i,ls=':',color='black')

        ax[0].set_xlabel('$r$ / m')
        ax[0].set_ylabel('$u_z$ / mm + $z$ / m')
        ax[0].set_yticks([0,-1,-2,-3,-4,-5])
        ax[0].set_yticklabels([0,1,2,3,4,5])
        ax[0].set_xlim(0,5)
        ax[0].spines['bottom'].set_position(('data', 0))
        ax[0].xaxis.set_label_position('top') 
        ax[0].xaxis.tick_top()
        #
        z = np.linspace(0.1,5,100)
        ax[1].plot(z,u3D(0,z,E,nu,F)*1e3,label='3D solution')
        ax[1].plot(z,u1D(0,z,E,nu,F)*1e3,label='$\\epsilon_{xx}=\\epsilon_{yy} = 0$',ls='--')
        ax[1].set_xlabel('$z$ / m')
        ax[1].set_ylabel('$u_z$ / mm')
        ax[1].set_xlim(left=0)
        ax[1].set_ylim(bottom=0)
        ax[1].legend()
        fig.tight_layout();

In [ ]:
def u3D_Rechteck(t,a,b,E=100e6,nu=0.3,s0=100.e3):
    Rad = np.sqrt(a**2+b**2+t**2)
    fak1 = s0*(1-nu**2)/(np.pi * E)
    fak2 = s0*(1-nu-2*nu**2)/(2*np.pi * E)
    t1 = a*np.log( (b + np.sqrt(a**2+b**2)) * np.sqrt(a**2+t**2) / (a*(b+Rad)) )
    t2 = b*np.log( (a + np.sqrt(a**2+b**2)) * np.sqrt(b**2+t**2) / (b*(a+Rad)) )
    t3 = t * np.arctan(a*b/(t*Rad))
    return fak1 * (t1 + t2) + fak2 * t3

In [ ]:
def interactive_rect_settlement():
    @interact(#c=widgets.BoundedFloatText(value=0.3,min=0, description='$c$ / MPa'),
               E=widgets.BoundedFloatText(value=100, min=1e-3, max=1e3, description='$E$ / MPa'),
               s0=widgets.BoundedFloatText(value=100, min=1, max=1e4, description='$\\sigma_0$ / kPa'),
               a=widgets.BoundedFloatText(value=3, min=0.1, max=10, description='$a$ / m'),
               b=widgets.BoundedFloatText(value=2, min=0.1, max=10, description='$b$ / m')
    )
    def plot_MC_states(E,s0,a,b):
        E *= 1e6
        s0 *=1e3
        t = np.linspace(0.1,100,1000) # m
        sns.set_palette("rocket")
        fig, ax = plt.subplots(figsize=(12,8))
        for nu in [0.0,0.1,0.2,0.3,0.4,0.5]:
            ax.plot(t,u3D_Rechteck(t,a,b,E,nu,s0)*1e3,label='$\\nu = %.1f$' %nu)
        ax.set_xlabel('$t$ / m')
        ax.set_ylabel('$s$ / mm')
        ax.set_xlim(left=0)
        ax.set_ylim(bottom=0)
        ax.legend()
        fig.tight_layout();